In [11]:
import torch

In [12]:
torch.backends.mps.is_built()
torch.backends.mps.is_available()

True

In [33]:
import pandas as pd
import numpy as np
PATH = '/Users/soyoung/ml_project/test_proj/따릉이/train.csv'
data = pd.read_csv(PATH)

In [34]:
data.dropna(inplace=True)

In [35]:
y = data.pop('count')
X = data

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [37]:
from pytorch_tabnet.tab_model import TabNetClassifier,TabNetRegressor

In [38]:
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy().reshape(-1,1), y_test.to_numpy().reshape(-1,1)

In [43]:
X_train

array([[1.165e+03, 1.800e+01, 1.690e+01, ..., 5.000e-02, 7.300e+01,
        5.700e+01],
       [1.480e+03, 1.300e+01, 2.550e+01, ..., 5.700e-02, 3.700e+01,
        2.200e+01],
       [3.340e+02, 1.600e+01, 2.750e+01, ..., 6.700e-02, 4.500e+01,
        2.700e+01],
       ...,
       [1.190e+03, 1.500e+01, 2.680e+01, ..., 6.700e-02, 8.600e+01,
        4.600e+01],
       [1.259e+03, 6.000e+00, 1.260e+01, ..., 4.000e-03, 3.200e+01,
        2.000e+01],
       [1.812e+03, 1.200e+01, 1.810e+01, ..., 4.600e-02, 3.400e+01,
        1.700e+01]])

In [48]:
from pytorch_tabnet.pretraining import TabNetPretrainer

# TabNetPretrainer
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax' # "sparsemax"
)

unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_test],
    pretraining_ratio=0.8,
    batch_size=16

)

clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    from_unsupervised=unsupervised_model
)


epoch 0  | loss: 58.4933 | val_0_unsup_loss_numpy: 11.190170288085938|  0:00:01s
epoch 1  | loss: 4.74015 | val_0_unsup_loss_numpy: 3.665410041809082|  0:00:03s
epoch 2  | loss: 2.68062 | val_0_unsup_loss_numpy: 2.7881999015808105|  0:00:04s
epoch 3  | loss: 2.4371  | val_0_unsup_loss_numpy: 2.2457900047302246|  0:00:05s
epoch 4  | loss: 2.17924 | val_0_unsup_loss_numpy: 2.090590000152588|  0:00:07s
epoch 5  | loss: 2.39277 | val_0_unsup_loss_numpy: 2.042479991912842|  0:00:08s
epoch 6  | loss: 2.53653 | val_0_unsup_loss_numpy: 2.3040199279785156|  0:00:10s
epoch 7  | loss: 2.54658 | val_0_unsup_loss_numpy: 2.962480068206787|  0:00:11s
epoch 8  | loss: 2.31244 | val_0_unsup_loss_numpy: 2.021549940109253|  0:00:13s
epoch 9  | loss: 1.97286 | val_0_unsup_loss_numpy: 1.879140019416809|  0:00:14s
epoch 10 | loss: 2.10098 | val_0_unsup_loss_numpy: 1.9062199592590332|  0:00:16s
epoch 11 | loss: 2.01885 | val_0_unsup_loss_numpy: 1.8871599435806274|  0:00:17s
epoch 12 | loss: 2.29285 | val_0_u

ValueError: Data must be 1-dimensional